In [1]:
import netCDF4 as nc
import matplotlib.pyplot as plt
import base64
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

def get_extents(dataset, variable):
    # Extract the GeoTransform attribute
    geo_transform = dataset.variables['spatial_ref'].GeoTransform.split()
    print(geo_transform)

    # Convert GeoTransform values to float
    geo_transform = [float(value) for value in geo_transform]

    # Calculate the extents (four corners)
    min_x = geo_transform[0]
    pixel_width = geo_transform[1]
    max_y = geo_transform[3]
    pixel_height = geo_transform[5]

    # Get the actual dimensions of the raster layer
    n_rows, n_cols = dataset.variables[variable][0, :, :].shape

    # Calculate the coordinates of the four corners
    top_left = (min_x, max_y)
    top_right = (min_x + n_cols * pixel_width, max_y)
    bottom_left = (min_x, max_y + n_rows * pixel_height)
    bottom_right = (min_x + n_cols * pixel_width, max_y + n_rows * pixel_height)

    return top_left, top_right, bottom_left, bottom_right

def open_netcdf(file_path):
    # Open the NetCDF file
    dataset = nc.Dataset(file_path, mode='r')

    return dataset

def encode_raster_layer(dataset, variable):
    # Extract snow cover
    raster_layer = dataset.variables[variable][0, :, :]

    # Convert the array to bytes and encode the bytes in base64
    bytes = raster_layer.tobytes()
    base64_raster_layer = base64.b64encode(bytes).decode('utf-8')

    return raster_layer, base64_raster_layer

def decode_raster_layer(raster_layer, base64_raster_layer):
    # Decode the base64 string back to bytes
    decoded_bytes = base64.b64decode(base64_raster_layer)
    decoded_raster_layer = np.frombuffer(decoded_bytes, dtype=raster_layer.dtype).reshape(raster_layer.shape)

    return decoded_raster_layer

def open_encode(file_path, variable):
    # Open the NetCDF file
    dataset = open_netcdf(file_path)

    # Get the extents (four corners) coordinates
    top_left, top_right, bottom_left, bottom_right = get_extents(dataset, variable=variable)

    # Encode the raster layer
    raster_layer, base64_raster_layer = encode_raster_layer(dataset, variable=variable)

    # Decode the raster layer
    decoded_raster_layer = decode_raster_layer(raster_layer, base64_raster_layer)

    return raster_layer, base64_raster_layer, top_left, top_right, bottom_left, bottom_right

In [2]:
snow_layer_raster, snow_layer, top_left, top_right, bottom_left, bottom_right = open_encode(file_path='/mnt/c/Users/emgonz38/OneDrive - Arizona State University/ubuntu_files/netcdf_encode/input_data/Efficiency_high_resolution_Caesium/efficiency_snow_cover_highest_resolution.nc',
                                                 variable='Weekly_Snow_Cover')
resolution_layer_raster, resolution_layer, top_left, top_right, bottom_left, bottom_right = open_encode(file_path='/mnt/c/Users/emgonz38/OneDrive - Arizona State University/ubuntu_files/netcdf_encode/input_data/Efficiency_high_resolution_Caesium/efficiency_resolution_layer_highest_resolution.nc',
                                                       variable='Monthly_Resolution_Abs')

# snow_layer, top_left, top_right, bottom_left, bottom_right = open_encode(file_path='/mnt/c/Users/emgonz38/OneDrive - Arizona State University/ubuntu_files/netcdf_encode/input_data/Efficiency_resolution20_Optimization/efficiency_snow_cover.nc',
#                                                     variable='Day_CMG_Snow_Cover')
# resolution_layer, top_left, top_right, bottom_left, bottom_right = open_encode(file_path='/mnt/c/Users/emgonz38/OneDrive - Arizona State University/ubuntu_files/netcdf_encode/input_data/Efficiency_resolution20_Optimization/efficiency_resolution_layer.nc',
#                                                         variable='Monthly_Resolution_Abs')

['-113.91582164189471', '0.05000694540908461', '0.0', '49.763823284245625', '0.0', '-0.05001389274798555']
['-113.94166666666666', '0.008333333333333331', '0.0', '49.741666666666674', '0.0', '-0.008333333333333335']


In [3]:
plt.imsave('snow_raster_layer_high_resolution.png', snow_layer_raster, cmap='viridis')

In [4]:
plt.imsave('resolution_raster_layer_high_resolution.png', resolution_layer_raster, cmap='viridis')

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Plot the original snow cover
im0 = axes[0].imshow(raster_layer, cmap='viridis')
axes[0].set_title('Original Day CMG Snow Cover')
axes[0].set_xlabel('Longitude')
axes[0].set_ylabel('Latitude')

# Adjust the colorbar to be shorter
divider0 = make_axes_locatable(axes[0])
cax0 = divider0.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im0, cax=cax0, label='Snow Cover')
# fig.colorbar(axes[0].images[0], ax=axes[0], label='Snow Cover')

# Plot the decoded snow cover
im = axes[1].imshow(decoded_raster_layer, cmap='viridis')
axes[1].set_title('Decoded Day CMG Snow Cover')
axes[1].set_xlabel('Longitude')
axes[1].set_ylabel('Latitude')
# Adjust the colorbar to be shorter

divider = make_axes_locatable(axes[1])
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(im, cax=cax, label='Snow Cover')

plt.tight_layout()
plt.show()